In [127]:
import cv2
import numpy as np

In [128]:
def get_line_pixels_horizontal(image, x0, y0, x1, y1):
    # Implement the Bresenham's line algorithm
    points = []
    dx = abs(x1 - x0)
    dy = abs(y1 - y0)
    x, y = x0, y0
    sx = -1 if x0 > x1 else 1
    sy = -1 if y0 > y1 else 1
    if dx > dy:
        err = dx / 2.0
        while x != x1:
            if 0 <= x < len(image[0]) and 0 <= y < len(image):
                if 0 <= y-1 < len(image) and 0 <= y+1 < len(image):
                    points.append((x, y))
            err -= dy
            if err < 0:
                y += sy
                err += dx
            x += sx
    else:
        err = dy / 2.0
        while y != y1:
            if 0 <= x < len(image[0]) and 0 <= y < len(image):
                if 0 <= y-1 < len(image) and 0 <= y+1 < len(image):
                    points.append((x, y))
            err -= dx
            if err < 0:
                x += sx
                err += dy
            y += sy
    if 0 <= x < len(image[0]) and 0 <= y < len(image):
        if 0 <= y-1 < len(image) and 0 <= y+1 < len(image):
            points.append((x, y))

    # Get the pixel values
    pixels = [(image[y-1][x], image[y+1][x]) for x, y in points]

    return pixels

In [129]:
# Define a function to calculate color contrast between two pixels
def color_contrast(pixel1, pixel2):
    return pixel1 - pixel2

In [130]:
# Load the image
image = cv2.imread('cropped/AA AB-261-[0].jpg', 0)  # 0 loads the image in grayscale


In [131]:
# Define the line parameters (angle and starting point)
angle = -13  # Replace with your desired angle in degrees
x0, y0 = 0, 87  # Replace with your desired starting point

# Convert the angle to radians
angle = np.radians(angle)

# Calculate the end point of the line
x1 = int(x0 + 500 * np.cos(angle))
y1 = int(y0 + 500 * np.sin(angle))

In [132]:
image2 = image.copy()

cv2.line(image2, (x0, y0), (x1, y1), 255, 1)
cv2.line(image2, (x0, y0-1), (x1, y1-1), 0, 1)
cv2.line(image2, (x0, y0+1), (x1, y1+1), 0, 1)

cv2.imshow('image', image2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [133]:
all_horizontals = []
for angle in range(-15, 15):
    all_contrasts = []

    # print("Angle: ", angle)
    # Convert the angle to radians
    angle = np.radians(angle)

    for i in range(image.shape[0]):
        # Define the line parameters (angle and starting point)
        x0, y0 = 0, i  # Replace with your desired starting point

        # Calculate the end point of the line
        x1 = int(x0 + 500 * np.cos(angle))
        y1 = int(y0 + 500 * np.sin(angle))

        pixels = get_line_pixels_horizontal(image, x0, y0, x1, y1)
        pixels = np.array(pixels, dtype=np.int16)

        if len(pixels) == 0:
            continue

        total_contrast = pixels[:, 0] - pixels[:, 1]
        all_contrasts.append(total_contrast.mean())

    all_contrasts = np.array(all_contrasts)
    # print(all_contrasts.argmin(), all_contrasts.min())
    # print(all_contrasts.argmax(), all_contrasts.max())
    length = all_contrasts.shape[0]
    argmin = all_contrasts.argmin()
    min = all_contrasts.min()
    argmax = all_contrasts.argmax()
    max = all_contrasts.max()
    all_horizontals.append((argmin, min, argmax, max, max-min))

In [134]:
all_horizontals = np.array(all_horizontals)
best_lines = all_horizontals[all_horizontals[:, 4].argmax()]

# Define the line parameters (angle and starting point)
angle = -15 + all_horizontals[:, 4].argmax()
x0_left, y0_left = 0, int(best_lines[0])
x0_right, y0_right = 0, int(best_lines[2])

print("Angle: ", angle)
print(best_lines)

# Convert the angle to radians
angle = np.radians(angle)

# Calculate the end point of the lines
x1_top = int(x0_left + 500 * np.cos(angle))
y1_top = int(y0_left + 500 * np.sin(angle))
x1_bottom = int(x0_right + 500 * np.cos(angle))
y1_bottom = int(y0_right + 500 * np.sin(angle))

image2 = image.copy()

cv2.line(image2, (x0_left, y0_left), (x1_top, y1_top), 255, 1)
cv2.line(image2, (x0_left, y0_left-1), (x1_top, y1_top-1), 0, 1)
cv2.line(image2, (x0_left, y0_left+1), (x1_top, y1_top+1), 0, 1)

cv2.line(image2, (x0_right, y0_right), (x1_bottom, y1_bottom), 255, 1)
cv2.line(image2, (x0_right, y0_right-1), (x1_bottom, y1_bottom-1), 0, 1)
cv2.line(image2, (x0_right, y0_right+1), (x1_bottom, y1_bottom+1), 0, 1)

cv2.imshow('image', image2)
cv2.waitKey(0)
cv2.destroyAllWindows()

Angle:  4
[  7.         -75.14642857  64.          96.85357143 172.        ]
